In [77]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [78]:
%%bash
uname -a
whoami

edmFileUtil root://eoscms.cern.ch///store/express/Run2017C/ExpressPhysics/FEVT/Express-v1/000/299/593/00000/C0D83139-4A6E-E711-B1A9-02163E0137FC.root
edmFileUtil root://eoscms.cern.ch///store/express/Run2017C/ExpressPhysics/FEVT/Express-v1/000/299/593/00000/4269D74E-4A6E-E711-847B-02163E01A487.root

Linux vinavx3.cern.ch 3.10.0-514.10.2.el7.x86_64 #1 SMP Fri Mar 3 00:04:05 UTC 2017 x86_64 x86_64 x86_64 GNU/Linux
innocent
root://eoscms.cern.ch///store/express/Run2017C/ExpressPhysics/FEVT/Express-v1/000/299/593/00000/C0D83139-4A6E-E711-B1A9-02163E0137FC.root
root://eoscms.cern.ch///store/express/Run2017C/ExpressPhysics/FEVT/Express-v1/000/299/593/00000/C0D83139-4A6E-E711-B1A9-02163E0137FC.root (1 runs, 1 lumis, 1522 events, 2449095209 bytes)
root://eoscms.cern.ch///store/express/Run2017C/ExpressPhysics/FEVT/Express-v1/000/299/593/00000/4269D74E-4A6E-E711-847B-02163E01A487.root
root://eoscms.cern.ch///store/express/Run2017C/ExpressPhysics/FEVT/Express-v1/000/299/593/00000/4269D74E-4A6E-E711-847B-02163E01A487.root (1 runs, 3 lumis, 4077 events, 2088712010 bytes)


In [79]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
import time
from multiprocessing import Pool
from DataFormats.FWLite import Handle, Events

In [80]:
xrd = 'root://cms-xrd-global.cern.ch//'
tier0 = 'root://cms-xrd-tzero.cern.ch//' # 'file:/eos/cms/tier0'
tier2 = 'root://eoscms.cern.ch//' #'file:/eos/cms'
myTracking = 'root://eoscms.cern.ch///store/group/phys_tracking/vincenzo/run2017/'

def oldStuff() :
    files = [\
    '/store/data/Run2016H/JetHT/AOD/PromptReco-v2/000/283/408/00000/AE49FDAB-8398-E611-9B98-02163E014388.root',\
    '/store/data/Run2016H/JetHT/AOD/PromptReco-v2/000/283/408/00000/88FB6BA5-7B98-E611-B25E-FA163EE1CC08.root',\
    '/store/data/Run2017B/SingleMuon/AOD/PromptReco-v2/000/299/061/00000/5C4984B0-B86A-E711-837F-02163E01A792.root',
    '/store/data/Run2017B/SingleMuon/AOD/PromptReco-v2/000/299/067/00000/B4146406-F66A-E711-B9C9-02163E014389.root',\
            ]
    return map(lambda x : xrd+x,files)


def jetHT2017C() :
    files = [\
    '/store/data/Run2017C/JetHT/AOD/PromptReco-v1/000/299/443/00000/7C92F761-A270-E711-90A8-02163E014506.root',\
    '/store/data/Run2017C/JetHT/AOD/PromptReco-v1/000/299/649/00000/D0BA6C4A-DB71-E711-99BC-02163E013910.root',\
    '/store/data/Run2017C/JetHT/AOD/PromptReco-v1/000/299/592/00000/44735821-ED70-E711-A9F3-02163E019C7D.root',\
    '/store/data/Run2017C/JetHT/AOD/PromptReco-v1/000/299/478/00000/7C39B1F9-A970-E711-AC40-02163E0138D5.root',\
    '/store/data/Run2017C/JetHT/AOD/PromptReco-v1/000/299/480/00000/9276CDF3-B870-E711-9878-02163E019D8F.root',\
    '/store/data/Run2017C/JetHT/AOD/PromptReco-v1/000/299/478/00000/E2BB4D9E-A870-E711-8684-02163E0142DF.root',\
    '/store/data/Run2017C/JetHT/AOD/PromptReco-v1/000/299/420/00000/5EC981C1-426E-E711-9C05-02163E0135F2.root',\
            ]
    return map(lambda x : tier0+x,files)
    
def express2017C() :
    files = [\
    '/store/express/Run2017C/ExpressPhysics/FEVT/Express-v1/000/299/592/00000/1482C818-456E-E711-A4AC-02163E01A3E1.root',\
    '/store/express/Run2017C/ExpressPhysics/FEVT/Express-v1/000/299/593/00000/4CA2B934-716E-E711-B4F0-02163E01A3E5.root',\
    '/store/express/Run2017C/ExpressPhysics/FEVT/Express-v1/000/299/593/00000/523D4165-4A6E-E711-B186-02163E012A87.root',\
    '/store/express/Run2017C/ExpressPhysics/FEVT/Express-v1/000/299/593/00000/C0D83139-4A6E-E711-B1A9-02163E0137FC.root',\
    '/store/express/Run2017C/ExpressPhysics/FEVT/Express-v1/000/299/593/00000/C6AE7442-4A6E-E711-864C-02163E019D7E.root',\
    '/store/express/Run2017C/ExpressPhysics/FEVT/Express-v1/000/299/593/00000/4269D74E-4A6E-E711-847B-02163E01A487.root',\
    '/store/express/Run2017C/ExpressPhysics/FEVT/Express-v1/000/299/593/00000/2449BE69-4A6E-E711-BCD5-02163E0128FE.root',\
    '/store/express/Run2017C/ExpressPhysics/FEVT/Express-v1/000/299/593/00000/9C51DE1E-4A6E-E711-9701-02163E019E83.root',\
    '/store/express/Run2017C/ExpressPhysics/FEVT/Express-v1/000/299/649/00000/28D1AE00-2170-E711-96CA-02163E019D3B.root'\
    ]
    return map(lambda x : tier2+x,files)

def myFiles() :
    files = [\
             'JetHT_reco299592.root',\
             'JetHT_tkreco299592.root',\
             ]
    return map(lambda x : myTracking+x,files)

In [81]:
label = "generalTracks"
quality = "highPurity"

In [82]:
def runid(event,lumi) :
    id = event.object().id()
    a=event.getByLabel('scalersRawToDigi', lumi)
    return '{:d}:{:d}:{:.2e}'.format(int(id.run()),int(id.luminosityBlock()),lumi.product()[0].instantLumi())
def skip2Lumi(events,ls):
    for i in range(0,events.size()):
        a= events.to(i)
        id = events.object().id()
        if (int(id.luminosityBlock())==ls) : return i
    return events.size()

In [83]:
def loadTk(events,tracks,fe) :
    tkParsA = []
    tkHitsA = []
    for i in range(fe,events.size()):
      if (len(tkParsA) > 1000000) : break
      a= events.to(i)
      if (i%500==0) : 
        id = events.object().id()
        evid = '{:d}:{:d}:{:d}'.format(int(id.run()),int(id.luminosityBlock()), int(id.event()))
        print "Event", i , evid
      a=events.getByLabel(label, tracks)
      for tk in tracks.product() :
        if (not tk.quality(tk.qualityByName(quality))) : continue
        if (tk.pt()<0.7) : continue
        pattern = tk.hitPattern()
        if (pattern.numberOfValidHits()<8) : continue
        if (pattern.numberOfValidPixelHits()<1) : continue
        tkParsA.append([tk.eta(),tk.phi(),tk.pt(),pattern.numberOfValidHits(),pattern.numberOfValidPixelHits(),tk.ndof(),tk.chi2(),tk.originalAlgo()-4])
        tkHitsA.append([pattern.getTrackerLayerCase(0,1,1), pattern.getTrackerLayerCase(1,1,1),\
                        pattern.getTrackerLayerCase(0,1,2), pattern.getTrackerLayerCase(1,1,2),\
                        pattern.getTrackerLayerCase(0,2,1), pattern.getTrackerLayerCase(1,2,1)])

    print len(tkParsA)
    tkPars = np.array(zip(*tkParsA))
    print len(tkPars)
    tkHits = np.array(zip(*tkHitsA))
    print len(tkHits)
    return tkPars,tkHits

In [84]:
def processIt(fevents) :
    print fevents
    events = Events(fevents)
    tracks = Handle("std::vector<reco::Track>")
    lumi = Handle("std::vector<LumiScalers>")
    a= events.to(0)
    ls = int(events.object().id().luminosityBlock())
    if (ls==103) : ls=105
    fe = skip2Lumi(events,ls)
    a= events.to(fe)
    runid0 = runid(events,lumi)
    print runid0
    tkPars, tkHits = loadTk(events,tracks,fe)
    runidl = runid(events,lumi)
    print runidl
    return (runid0+'-->'+runidl, tkPars, tkHits)

In [ ]:
p = Pool(4)

root://cms-xrd-tzero.cern.ch///store/data/Run2017C/JetHT/AOD/PromptReco-v1/000/299/443/00000/7C92F761-A270-E711-90A8-02163E014506.root
root://cms-xrd-tzero.cern.ch///store/data/Run2017C/JetHT/AOD/PromptReco-v1/000/299/592/00000/44735821-ED70-E711-A9F3-02163E019C7D.root
root://cms-xrd-tzero.cern.ch///store/data/Run2017C/JetHT/AOD/PromptReco-v1/000/299/649/00000/D0BA6C4A-DB71-E711-99BC-02163E013910.root
root://cms-xrd-tzero.cern.ch///store/data/Run2017C/JetHT/AOD/PromptReco-v1/000/299/478/00000/7C39B1F9-A970-E711-AC40-02163E0138D5.root
299649:69:1.66e+04
Event 0 299649:69:64632179
299592:83:1.44e+04
Event 0 299592:83:79658055
299443:56:1.63e+04
Event 0 299443:56:16216001
299478:84:1.29e+04
Event 0 299478:84:119847131
Event 500 299592:83:80191770
Event 500 299649:69:63947887
Event 500 299443:56:17792575
Event 1000 299592:83:79689730
Event 1000 299649:69:64938580
Event 1500 299592:83:80089575
Event 1000 299443:56:17316657
Event 1500 299649:70:66383596
Event 500 299478:84:120066050
Event 20

In [ ]:
files =  jetHT2017C() # express2017C() # myFiles() $oldStuff() #
t = -time.time()
ret = p.map(processIt,files)
t +=time.time()
print t
runid0, tkPars, tkHits = zip(*ret)
print runid0
print len(tkHits)

In [ ]:
p.terminate()

In [ ]:
def eff(var,tkPars,tkHits) :
    hp = np.greater(tkPars[2],0.7)
    valL2 = np.equal(tkHits[2],0)
    valF1 = np.equal(tkHits[4],0)
    hp = hp&(valL2) # |valF1)
    nbins = 62
    yv,x = np.histogram(tkPars[var],np.linspace(-3.1,3.1,nbins),weights=1.*hp*np.equal(tkHits[0],0)) 
    yl,x = np.histogram(tkPars[var],np.linspace(-3.1,3.1,nbins),weights=1.*hp*np.equal(tkHits[0],1)) 
    #yi,x = np.histogram(tkPars[var],np.linspace(-3.1,3.1,nbins),weights=1.*hp*np.equal(tkHits[0],2)) 
    ym,x = np.histogram(tkPars[var],np.linspace(-3.1,3.1,nbins),weights=1.*hp*np.equal(tkHits[1],1)) 
    ya=yv+ym # +yl
    # y2,x = np.histogram(tkPars[var],np.linspace(-3.1,3.1,nbins),weights=1.*hp)
    x.resize(len(yv))
    return x,yv/ya

In [ ]:
eta=0
phi=1
pt=2

In [ ]:
bha = map(eff,[phi]*len(tkPars),tkPars, tkHits)
print len(bha)

In [ ]:
i=0
for b in bha:
    plt.step(b[0],b[1],where='post',label=runid0[i])
    plt.xlim(-3.1, 3.1)
    plt.ylim(0.5, 1.05)
    plt.grid(True)
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
#    plt.legend(loc='upper right')
    plt.ylabel('eff')
    i+=1
    plt.show()
